## Download Dataset

you can manually download the dataset with link : https://drive.google.com/file/d/1wYVdCSW8ipjJ2Zr8zoqe1w1LGc5jhufW/view?usp=sharing

and insert the dataset on working directory

In [ ]:
# Download the dataset from MEGA will takes 15-20 Minutes on Google Colab
!pip install mega.py

from mega import Mega

mega = Mega()

m = mega.login()

m.download_url('https://mega.nz/file/bBoSSRra#szRg4E4glGBVUryt5-BI5GvaRymLYPYsIMxbH9Soflc')

In [ ]:
import zipfile

local_zip = 'Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('')
zip_ref.close()

## Import Required Library

In [ ]:
import os

from __future__ import absolute_import, division, print_function, unicode_literals

from numpy.random import seed
seed(101)

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import confusion_matrix

from matplotlib import pyplot as plt
%matplotlib inline
print("Running TensorflowVersion: " + str(tf.__version__))

## Select appropriate distribution strategy for hardware

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices('GPU')

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU: " + str(tpu.master()))
elif len(gpus) > 0:
    strategy = tf.distribute.MirroredStrategy(gpus)
    print("Running on ",len(gpus)," GPU(s)")
else:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

## Setup Strategy Based on Hardware

In [ ]:
train_path = "Dataset/Train"
val_path = "Dataset/Validation"
test_path = "Dataset/Test"

train_images = [image for dir,_,sublist in os.walk(train_path) for image in sublist]
val_images = [image for dir,_,sublist in os.walk(val_path) for image in sublist]
test_images = [image for dir,_,sublist in os.walk(test_path) for image in sublist]
num_train_images = len(train_images)
num_val_images = len(val_images)
num_test_images = len(test_images)

IMAGE_SIZE = 224 
EPOCHS = 10

#CLASSES = ['Boat', 'Bus', 'Car', 'Cat', 'Flower', 'Horse']
multi = 16 

#Learning rate scheduling variables
num_units = strategy.num_replicas_in_sync
if num_units == 8:
    BATCH_SIZE = multi * num_units
    VALIDATION_BATCH_SIZE = multi * num_units
    start_lr = 0.00001
    min_lr = 0.00001
    max_lr = 0.00005 * num_units
    rampup_epochs = 8
    sustain_epochs = 0
    exp_decay = 0.8
elif num_units == 1:
    BATCH_SIZE = multi
    VALIDATION_BATCH_SIZE = multi
    start_lr = 0.00001 #0.00001
    min_lr = 0.00001 #0.00001
    max_lr = 0.0002 #0.0002
    rampup_epochs = 8 #8
    sustain_epochs = 0
    exp_decay = 0.8
else:
    BATCH_SIZE = int(multi/2) * num_units
    VALIDATION_BATCH_SIZE = int(multi/2) * num_units
    start_lr = 0.00001  
    min_lr = 0.00001
    max_lr = 0.00002 * num_units
    rampup_epochs = 11
    sustain_epochs = 0
    exp_decay = 0.8
    
train_steps = int(np.ceil(num_train_images/BATCH_SIZE))
val_steps = int(np.ceil(num_val_images/VALIDATION_BATCH_SIZE))

print("Total Training Images: " + str(num_train_images))
print("Total Validation Images: " + str(num_val_images))
print("Total Test Images: " + str(num_test_images))

print("Train Steps: " + str(train_steps))
print("Val steps: " + str(val_steps))

def display_training_curves(training,validation,title,subplot):
    if subplot%10 == 1:
        plt.subplots(figsize = (10,10),facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    
def learningrate_function(epoch):
    if epoch < rampup_epochs:
        lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        lr = max_lr
    else:
        lr = (max_lr - min_lr) * exp_decay**(epoch - rampup_epochs - sustain_epochs) + min_lr
    return lr

def learning_rate_callback():
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch : learningrate_function(epoch),verbose = True)
    rng = [i for i in range(EPOCHS)]
    y = [learningrate_function(x) for x in range(EPOCHS)]
    plt.plot(rng,y)
    return lr_callback

## Define Training, Validation, Test Dataset through Folder

In [ ]:
datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_datagen = datagen.flow_from_directory(train_path, 
                                            target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                            batch_size = BATCH_SIZE, 
                                            class_mode = 'categorical')

val_datagen = datagen.flow_from_directory(val_path,
                                          target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                          batch_size = VALIDATION_BATCH_SIZE,
                                          class_mode = 'categorical')

test_datagen = datagen.flow_from_directory(test_path,
                                            target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                            batch_size = 1, 
                                            class_mode = 'categorical',
                                            shuffle = False)

In [ ]:
CLASSES = list(train_datagen.class_indices.keys())
print(CLASSES)

## Define Model Layers

In [ ]:
with strategy.scope():        
    base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False) 
    base_model.trainable = True
    
    model = Sequential([
        base_model,       
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        Dense(6, activation = 'softmax', dtype= tf.float32)
    ])
    
    model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
    )
    
model.summary()

lr_callback = learning_rate_callback()

## Train the model (you can skip this step if only load the existing model)

you can manually download existing model (the exact model are used on the project) and insert on working directory with following link : https://drive.google.com/file/d/188gdv5nLEQfFAnCqHSmq9bX6FRkh34E_/view

In [ ]:
filepath = "model.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') #,mode='max'

callbacks_list = [checkpoint,lr_callback]

hist = model.fit(train_datagen,
                             steps_per_epoch = train_steps,
                             validation_data = val_datagen,
                             validation_steps = val_steps,
                             epochs = EPOCHS,
                             verbose = 1,
                             callbacks = callbacks_list)



val_loss, val_acc = model.evaluate(val_datagen,steps = num_val_images)

print('val_loss:', val_loss)
print('val_acc:', val_acc)

display_training_curves(hist.history['loss'], hist.history['val_loss'], 'loss', 211) 
display_training_curves(hist.history['accuracy'], hist.history['val_accuracy'], 'accuracy', 212)

## Load Trained Model

In [ ]:
model = load_model("model.h5")
model.summary()

val_loss, val_acc = model.evaluate(val_datagen,steps = num_val_images)

print('val_loss:', val_loss)
print('val_acc:', val_acc)

## Test the accuracy using test dataset

In [ ]:
predictions = model.predict(test_datagen, steps= num_val_images, verbose=1)

df_preds = pd.DataFrame(predictions,columns=CLASSES)
#df_preds.head()
y_true = val_datagen.classes

from sklearn.metrics import classification_report

# Generate a classification report

# For this to work we need y_pred as binary labels not as probabilities
y_pred_binary = predictions.argmax(axis=1)

report = classification_report(y_true, y_pred_binary, target_names=CLASSES)

print(report)